# sklearn IRIS
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=benchmark&ea=bentoml-benchmark&dt=sklearn-iris-bentoml)

Before we start

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
import bentoml
NAME = 'sklearn_iris'

# load data & train model

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

    
# add parameters for tuning
num_estimators = 100

# train the model
model = RandomForestRegressor(n_estimators=num_estimators)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('predictions: ', predictions)

# log model performance 
mse = mean_squared_error(y_test, predictions)
print("  mse: %f" % mse)

predictions:  [2.   1.   0.   1.   1.55 0.   1.15 1.   0.   1.   1.99 1.   0.   2.
 0.   1.9  2.   2.   0.   0.   1.   2.   1.01 1.27 1.55 1.9  1.   1.
 2.   2.  ]
  mse: 0.092020


In [4]:
%%writefile {NAME}.py

import bentoml
from bentoml.artifact import SklearnModelArtifact
from bentoml.handlers import DataframeHandler


@bentoml.env()
@bentoml.artifacts([SklearnModelArtifact('model')])
class BentoSvc(bentoml.BentoService):
    @bentoml.api(DataframeHandler)
    def predict(self, inputs):
        outputs = self.artifacts.model.predict(inputs.to_numpy())
        return outputs


Overwriting sklearn_iris.py


In [5]:
from sklearn_iris import BentoSvc

bento_svc = BentoSvc()
bento_svc.pack("model", model)
saved_path = bento_svc.save()
print(saved_path)

[2020-05-14 06:27:32,352] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-14 06:27:32,700] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.4.9+228.g3fbe5b9.dirty
creating BentoML-0.4.9+228.g3fbe5b9.dirty/BentoML.egg-info
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml/artifact
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml/bundler
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml/cli
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml/clipper
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml/configuration
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml/configuration/__pycache__
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml/deployment
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml/deployment/aws_lambda
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml/deployment/sagemaker
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml/handlers
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml/marshal
creating BentoML-0.4.9+228.g3fbe5b9.dirty/bentoml/migrations
creating BentoML-0.4.9+228.g3fb

# Build & Run Bento Service in Docker

In [6]:
from bentoml.utils import detect_free_port
PORT = detect_free_port()
print(PORT)

60251


In [6]:
!cd {saved_path}
IMG_NAME = saved_path.split('/')[-1].lower()
!docker build -t {IMG_NAME} {saved_path}

!docker run -itd -p {PORT}:5000 --cpus 1 -e FLAGS="--workers 1 --enable-microbatch" {IMG_NAME}:latest
# Optional: set your prefer PYPI mirror
# --build-arg PIP_TRUSTED_HOST=192.168.138.2 \
# --build-arg PIP_INDEX_URL=http://192.168.138.2/simple \

Sending build context to Docker daemon  592.4kB
Step 1/18 : FROM continuumio/miniconda3:4.8.2
 ---> b4adc22212f1
Step 2/18 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 53fb2c825bc1
Step 3/18 : RUN conda install pip numpy scipy       && pip install gunicorn
 ---> Using cache
 ---> b599776a4924
Step 4/18 : COPY . /bento
 ---> b2fa64293d1a
Step 5/18 : WORKDIR /bento
 ---> Running in f1d8ec7b6453
Removing intermediate container f1d8ec7b6453
 ---> 5ca90779ed9a
Step 6/18 : RUN if [ -f /bento/setup.sh ]; then /bin/bash -c /bento/setup.sh; fi
 ---> Running in 97991dc69c78
Removing intermediate container 97991dc69c78
 ---> e56172dd754f
Step 7/18 : RUN conda env update -n base -f /bento/environment.yml
 ---> Running in 1bad1a9a8bca
Solving environment: ...working... done

urllib3-1.25.8       | 169 KB    | ########## | 100% 
scipy-1.2.1          | 

In [7]:
print(f"bentoml serve-gunicorn {saved_path} --port {PORT} --workers 1 --enable-microbatch")
server_url = f"http://127.0.0.1:{PORT}/predict"
print(server_url)

bentoml serve-gunicorn /home/ec2-user/bentoml/repository/BentoSvc/20200514062732_591FCE --port 60251 --workers 1 --enable-microbatch
http://127.0.0.1:60251/predict


# Test with requests

In [2]:
server_url = 'http://127.0.0.1:60251/predict'
import json
import requests
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

headers = {"content-type": "application/json"}
raw_data = X_test
data = pd.DataFrame(raw_data).to_json()

json_response = requests.post(server_url,
                              data=data, headers=headers)
print(json_response)
print(json_response.json())

<Response [200]>
[2.0, 1.0, 0.0, 1.0, 1.55, 0.0, 1.15, 1.0, 0.0, 1.0, 1.99, 1.0, 0.0, 2.0, 0.0, 1.9, 2.0, 2.0, 0.0, 0.0, 1.0, 2.0, 1.01, 1.27, 1.55, 1.9, 1.0, 1.0, 2.0, 2.0]


# Benchmark

In [3]:
import pandas as pd
import json
import copy
import random


def get_request_producer():

    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    iris = datasets.load_iris()
    x = iris.data[:, 2:]
    y = iris.target

    url = server_url
    method = "POST"
    headers = {"content-type": "application/json"}
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=7)
    base_data = pd.DataFrame(X_test[:1]).to_dict()

    def _gen_data():
        raw = copy.deepcopy(base_data)
        raw[0][0] += random.random() / 10000
        raw[1][0] += random.random() / 10000
        data = json.dumps(raw)
        return url, method, headers, data

    return _gen_data

# test
get_request_producer()()

('http://127.0.0.1:60251/predict',
 'POST',
 {'content-type': 'application/json'},
 '{"0": {"0": 5.100077258613964}, "1": {"0": 1.8000226415345306}}')

In [4]:
import os


def init_file(file_name):
    if os.path.exists(LOG_FILE):
        return
    with open(file_name, "a") as lf:
        lf.write('"model name","test_users","total succ","succ/sec","avg resp time",'
                 '"total fail","fail/sec","avg fail resp time"')
        lf.write('\n')


def log_result(b, name, file_name, test_user):
    init_file(file_name)
    self = b.stat

    result = []
    result.append(name)
    result.append(test_user)
    result.append(self.success)
    result.append(self.success / max(self.sess_time, 1))
    result.append(sum(self.succ_times) / max(self.success, 1))

    result.append(self.fail)
    result.append(self.fail / max(self.sess_time, 1))
    result.append(sum(self.exec_times) / max(self.fail, 1))
    
    result.append((1 - self.client_busy / max(self.req_total, 1)))

    log_str = ','.join(str(r) for r in result)
    with open(file_name, "a") as lf:
        lf.write(log_str)
        lf.write('\n')

        
LOG_FILE = 'benchmark_result.csv'

In [8]:
from bentoml.utils.benchmark import BenchmarkClient

import asyncio

for TEST_USER in range(100, 1600, 100):
    b = BenchmarkClient(get_request_producer(), lambda: 1, timeout=0.32)
    await b._start_session(60, TEST_USER, TEST_USER)
    log_result(b, f'{NAME}', LOG_FILE, TEST_USER)
    b.killall()
    await asyncio.sleep(15)

======= Session started! =======

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │       0 │        0 │             nan │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │       0 │        0 │             nan │                   │
╘══════════╧═════════╧══════════╧═════════════════╧═══════════════════╛
------ 100 users spawned ------

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │     192 │       95 │       0.0188383 │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │       0 │        0 │     nan         │                  